# Simple QM/MM Simulation of the Claisen Rearrangment Reaction

In this tutorial, we will perform a QM/MM simulation of the claisen rearrangement reaction in Amber.

We will study the reaction under implicit solvent, where all ligand atoms included into the QM region, defined by the PM3 semi-empirical method. Umbrella Sampling will be used to explore the reaction coordinate and MBAR to to obtain the free energy profile.


```{figure} ../../_static/claisen.mp4
---
width: 50%
---
Trajectory of the claisen rearrangement (shown as ball-and-stick) viewed on VMD.
```

In this tutorial, we will:

* Run QM/MM Simulations on a Supercomputer (Oscer/Pete)
* Prepare a new project folder "claisen" and discuss directory structure/organization
* Use IQmol to model ligand
* Generate parameters for the ligand with `antechamber`, `parmchk2`, and `tleap`
* Prepare Umbrella Windows
* Create a "mbar" conda environment for the free energy profile analysis

````{note}
- **This tutorial requires a supercomputing account!**
- My example is with Pete, so some modules might differ
- Download my files for reference: [claisen.tar.bz2](../../assets/claisen.tar.bz2 "download")
````

## IQmol

Use IQmol to make a model of allyl vinyl ether, and save the file as `allyl_vinyl_ether.pdb`, remember to:

1. Add hydrogens
2. Minimize energy
3. Save as `.pdb`

The file is saved on my Desktop, we will upload the file to Pete at a later step!

```{figure} ../../_static/iqmol_claisen.mov
---
width: 50%
---
Using IQmol to make the allyl vinyl ester PDB file.
```

Login to Pete with `ssh`:

```bash
ssh USERNAME@pete.hpc.okstate.edu  # Change `USERNAME` to your username
```

`cd` to your `/scratch`  and make a new project folder called "claisen":

```bash
cd /scratch/USERNAME/  # Change `Username` to your username
mkdir -p claisen/pm3   # Make project folder, and method folder
cd claisen/pm3         # Go there
```

## Directory Structure & Organization

I recommend to organize the project folders like this:

Adopted from Dr. Xiaoliang Pan (hehe)

## Input Directory

Make the `input` directory. This is where we will store the PDB files and generate Amber inputs.

```bash
mkdir input # Make folder
cd input    # Go into folder
pwd         # Print working directory (Copy this output for scp)
```

Now, copy the PDB file of allyl vinyl ether you made with IQmol into the `input` folder! This will look different depending where you saved files, but open terminal on your local computer and use `scp` to upload the file to this directory (copied from `pwd`):

```bash
scp /path/to/allyl_vinyl_ether.pdb USERNAME@pete.hpc.okstate.edu:/scratch/USERNAME/claisen/input
```

Before moving on, check if the file transferred correctly. In the terminal window that is logged into Pete, check to see if the PDB file is in the `input/` folder.

### Generate Parameters for Allyl Vinyl Ether

To make a Amber topology and coordinate file for a small molecule, we need to:

1. Add hydrogens
2. Get charges with `antechamber`
3. Write parameters with `parmchk2`
4. Make topology and coordinate files with `tleap`

I prepared the script, `generate.slurm`, to help with this. 

```shell
#!/bin/bash
#SBATCH -p express
#SBATCH -t 00:10:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --output=%j.out
#SBATCH --error=%j.err
#SBATCH --job-name=antechamber

date

source /scratch/van/Scripts/pete_setqmmm.sh

name="allyl_vinyl_ether"

antechamber -fi pdb -fo mol2 -i ${name}.pdb -o ${name}.mol2 -c bcc -pf y -nc 0
parmchk2 -i ${name}.mol2 -f mol2 -o ${name}.frcmod

cat <<_EOF > tleap.in
source leaprc.gaff
source leaprc.water.tip3p 

loadamberparams allyl_vinyl_ether.frcmod 

ligand = loadmol2 allyl_vinyl_ether.mol2

solvatebox ligand TIP3PBOX 10.0 iso 0.8

saveamberparm ligand step3_pbcsetup.parm7 step3_pbcsetup.rst7
savepdb ligand step3_pbcsetup.pdb

quit
_EOF

tleap -sf tleap.in

date
```

Copy of this script to the `inputs/` folder and submit the job to Slurm:

```bash
sbatch generate.slurm
```

After a few seconds, you should see a few new files!

### Restraint Files

A restraint file (`cv.rst`) contains parameters for harmonic restraints in the modelling of a reaction mechanism. The general notation looks like this:

```shell
# r1 - r2 (CC - CO bond)
 &rst
  iat=3,4,1,6,
  rstwt=1.,-1.,
  r1=-99, r2=__REST__, r3=__REST__, r4=99,
  rk2=150.0, rk3=150.0,
 &end
```

Each parameter is described by:

- `# r1 - r2 (CC - CO bond)` Comment describing the restraint. In this case, we are doing restraining the difference between the first distance (r1) and the second distance (r2). 

- `iat=3,4,1,6,` Atom numbers for the restraint. The first distance (r1) is the length between atoms 3 and 4. The second distance (r2) is the length between atoms 1 and 6.

- `rstwt=1.,-1.,` Indicates we want to take the difference between r1 and r2.

- `r1=-99, r2=__REST__, r3=__REST__, r4=99,` The shape of our biasing potential. `__REST__` is a placeholder which we will modify later.

- `rk2=150.0, rk3=150.0,` Force constant in kcal/mol.

Make a copy of the `cv.rst` file and save it in the `input` folder.

```{important}
You do not need to modify it yet!!
```

## QM/MM MD Input files

2 MD input files are used. The first file (step5.00_equilibration.mdin) is used for to generate the initial pathway, and only runs for a few picoseconds. The second file (step6.00_equilibration.mdin) is used for additional sampling (+10 ps).

1. step5.00_equilibration.mdin
2. step6.00_equilibration.mdin

The general format of a QM/MM MD input file differs from classical MD with the `&qmmm` section.

::::{tab-set}
:::{tab-item} step5.00_equilibration.mdin
```shell
A NVT simulation for common production-level simulations
 &cntrl
    imin=0,        ! No minimization
    irest=1,       ! This IS a restart of an old MD simulation
    ntx=5,         ! So our inpcrd file has velocities

    ! Boundary conditions
    ntb=0,         ! Non-Periodic 
    ntp=0,         ! No pressure control

    ! Temperature control
    ntt=3,         ! Langevin Dynamics
    gamma_ln=5.0,  ! Friction coefficient (ps^-1)
    temp0=300.0,   ! Target temperature
    ig=-1,         ! Random number seed

    ! Potential energy control
    cut=999.0,      ! nonbonded cutoff, in Angstroms

    ! MD settings
    nstlim=500,   ! 1 ps total
    dt=0.001,      ! time step (ps)

    ! SHAKE
    ntc=1,         ! Constrain bonds containing hydrogen
    ntf=1,         ! Do not calculate forces of bonds containing hydrogen
    tol=0.000001,  ! Shake tolerance

    ! Control how often information is printed
    ntpr=100,      ! Print energies every 100 steps
    ntwx=100,      ! Print coordinates every 100 steps to the trajectory
    ntwr=100,      ! Print a restart file every 5K steps (can be less frequent)
    ntxo=2,        ! Write NetCDF format
    ioutfm=1,      ! Write NetCDF format (always do this!)

    ! Restraints
    nmropt=1,      ! Turn on restraints

    ! QM/MM
    ifqnt=1,       ! Turn on QM/MM
 /


 &qmmm
    ! QM atoms
    qmmask=':1'    ! Amber residue mask for QM atoms

    ! QM settings
    qm_theory='PM3', ! Semiempirical method
    qmcharge=0,    ! Charge of QM subsystem

    ! Shake
    qmshake=0,     ! Use Shake for QM atoms

    ! Potential energy control
    qmcut=999.0,    ! Cutoff for QM/MM electrostatic interactions
    writepdb=1,    ! Check QM atoms
 /

 &wt type='DUMPFREQ', istep1=10 /
 &wt type='END' /
 DISANG=cv.rst
 DUMPAVE=step5.00_equilibration.cv
```
:::
::::{tab-item} step6.00_equilibration.mdin
```shell
A NVT simulation for common production-level simulations
 &cntrl
    imin=0,        ! No minimization
    irest=1,       ! This IS a restart of an old MD simulation
    ntx=5,         ! So our inpcrd file has velocities

    ! Boundary conditions
    ntb=0,         ! Non-Periodic
    ntp=0,         ! No pressure control

    ! Temperature control
    ntt=3,         ! Langevin Dynamics 
    gamma_ln=5.0,  ! Friction coefficient (ps^-1)
    temp0=300.0,   ! Target temperature
    ig=-1,         ! Random number seed

    ! Potential energy control
    cut=999.0,      ! nonbonded cutoff, in Angstroms

    ! MD settings
    nstlim=10000,   ! 1 ps total
    dt=0.001,      ! time step (ps)

    ! SHAKE
    ntc=1,         ! Constrain bonds containing hydrogen
    ntf=1,         ! Do not calculate forces of bonds containing hydrogen
    tol=0.000001,  ! Shake tolerance

    ! Control how often information is printed
    ntpr=100,      ! Print energies every 100 steps
    ntwx=100,      ! Print coordinates every 100 steps to the trajectory
    ntwr=100,     ! Print a restart file every 5K steps (can be less frequent)
!   ntwv=-1,       ! Uncomment to also print velocities to trajectory
!   ntwf=-1,       ! Uncomment to also print forces to trajectory
    ntxo=2,        ! Write NetCDF format
    ioutfm=1,      ! Write NetCDF format (always do this!)

    ! Restraints
    nmropt=1,      ! Turn on restraints

    ! QM/MM
    ifqnt=1,       ! Turn on QM/MM
 /

 &ewald
    dsum_tol=0.000001,
 /

 &qmmm
    ! QM atoms
    qmmask=':1'

    ! QM settings
    qm_theory='PM3',
    qmcharge=0,

    ! Shake
    qmshake=0,     ! Use Shake for QM atoms
    ! Potential energy control
    qmcut=999.0,    ! Cutoff for QM/MM electrostatic interactions
    writepdb=1,    ! Check QM atoms
 /

 &wt type='DUMPFREQ', istep1=10 /
 &wt type='END' /
 DISANG=cv.rst
 DUMPAVE=step6.00_equilibration.cv
```
:::
::::

## Preparing Umbrella Windows

Go back one directory to the `pm3/` folder

```bash
cd ../ # Return back one directory
pwd    # Check where you are
```

The script `gen_inputs.in` will help you prepare windows for umbrella sampling. This includes the modification in  `cv.rst` file so that each window has a different restraint value.

We will have 21 windows corresponding to restraint values -2.00 to 2.00 Å.

```shell
#!/bin/bash

mkdir -p 00
cd 00/
ln -sf ../input/step3_pbcsetup.parm7
ln -sf ../input/step6.00_equilibration.mdin .
sed -e 's/irest=1/irest=0/;s/ntx=5/nts=1,/' ../input/step5.00_equilibration.mdin > step5.00_equilibration.mdin
cd ..

for i in `seq -w 1 20`; do
mkdir -p $i
cd $i
ln -sf ../input/step3_pbcsetup.parm7
ln -sf ../input/step5.00_equilibration.mdin .
ln -sf ../input/step6.00_equilibration .
cd ..
done

cp ../input/step3_pbcsetup.rst7 00/step5.00_equilibration_inp.rst7

n=-2.0
for i in `seq -f"%02g" 0 20`
do
    nn=$(printf "%.2f" "$n")
    sed "s/__REST__/${nn}/g" input/cv.rst > ${i}/cv.rst
    n=`echo $n + 0.20 | bc`
done
```

Run the script by:

```bash
bash gen_inputs.in
```

## Running QM/MM Simulation

We need to run QM/MM simulations of the inital pathway first (step5.00_equilibration). 

Submit the, `runqmmm1.slurm`, script to Slurm.

::::{tab-set}
:::{tab-item} runqmmm1.slurm
```shell
#!/bin/bash
#SBATCH --partition=express
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --output=%j.out
#SBATCH --error=%j.err
#SBATCH --time=00:30:00
#SBATCH --job-name=run1

date

source /scratch/van/Scripts/pete_setqmmm.sh

SANDER="srun -n 16 sander.MPI"

init="step3_pbcsetup"

istep="step5.00_equilibration"

for i in `seq 0 20`; do
    printf -v j "%02d" $i
    cd $j
    if [ ${i} -eq 0 ]; then
    $SANDER -O -i ${istep}.mdin -o ${istep}.mdout -p ${init}.parm7 -c ${istep}_inp.rst7 -r ${istep}.ncrst -x ${istep}.nc -inf ${istep}.mdinfo
    else
    $SANDER -O -i ${istep}.mdin -o ${istep}.mdout -p ${init}.parm7 -c ${istep}_inp.ncrst -r ${istep}.ncrst -x ${istep}.nc -inf ${istep}.mdinfo
    fi
    printf -v j "%02d" $(($i + 1))
    cp ${istep}.ncrst ../${j}/${istep}_inp.ncrst
    cd ..
done

date

```
:::
::::{tab-item} runqmmm2.slurm
```shell
#!/bin/bash
#SBATCH --partition=express
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --output=%j.out
#SBATCH --error=%j.err
#SBATCH --time=00:30:00
#SBATCH --job-name=run2

date

source /scratch/van/Scripts/pete_setqmmm.sh

SANDER="srun -n 16 sander.MPI"

init="step3_pbcsetup"

pstep="step5.00_equilibration"
istep="step6.00_equilibration"

for i in `seq -w 0 20`; do
    cd $i
    $SANDER -O -i ${istep}.mdin -o ${istep}.mdout -p ${init}.parm7 -c ${pstep}.ncrst -r ${istep}.ncrst -x ${istep}.nc -inf ${istep}.mdinfo
    cd ..
done

date

```
:::
::::

## Visualize the Trajectories